---
jupyter:
  jupytext:
    text_representation:
      extension: .md
      format_name: markdown
      format_version: '1.3'
      jupytext_version: 1.16.0
  kernelspec:
    display_name: Python 3
    language: python
    name: python3
---

<!-- #region id="9feccd08" -->
# Table des matières
1. [Génération des données](#Génération-des-données)
1. [Entraînement du modèle de régression](#Entraînement-du-modèle-de-régression)
1. [Calcul de la métrique $R^2$ pour les données d'entraînement et de test](#Calcul-de-la-métrique-$R^2$-pour-les-données-d'entraînement-et-de-test)
1. [Affichage des résultats](#Affichage-des-résultats)

# Attention!
Ne lancez pas l'exécution automatique du notebook en entier en cliquant sur le bouton **Tout exécuter**. L'exécution serait interrompue, car certaines cellules exigent une entrée de votre part!

Il faut simplement exécuter le notebook, une cellule à la fois, et entrer quelques lignes de code lorsque demandées. Il est inutile de sauter ces cellules pour aller aux suivantes car celles-ci ont justement besoin de votre input!

Importons d'abord les librairies nécessaires.
<!-- #endregion -->



In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# semence pour le générateur de nombres pseudoaléatoires
# permet d'assurer la reproductibilité des résultats
seed = 42

# initialisation du générateur à partir de la semence
np.random.seed(seed)



<!-- #region id="72f195fe" -->
Dans cet exemple, nous allons voir un modèle de régression où la réponse $y$ dépend de deux variables $x_{1}$ et $x_{2}$ pour lesquelles on sait qu'il y a interaction entre elles. Dans cet exemple, les données sont simulées afin de
simplifier l'analyse en plus de permettre au lecteur de modifier le nombre de données et le niveau de bruit
dans celles-ci afin de voir les résultats sur l'analyse.

On va montrer les différentes étapes de traitement de cet ensemble de données. C'est un exemple typique d'utilisation de
la régression dans l'analyse d'un jeu de données expérimentales.
<!-- #endregion -->

<!-- #region id="9e4e9a93" -->
## Génération des données

Nous allons générer un signal de la forme

$$y = a_{0} + a_{1}x_{1} + a_{2}x_{2} + a_{3}x_{1}x_{2}$$

où les valeurs de $x_1$ et $x_2$ sont aléatoirement distribuées dans le plan $(x_1, x_2)$ afin de simuler un échantillonnage aléatoire.

Nous allons ensuite additionner un bruit gaussien aux valeurs de $y$ afin de simuler des données expérimentales.

Commençons par générer des positions aléatoires en $x_{1}$ et $x_{2}$ dans les intervalles $[0, 10]$ et $[0, 15]$ respectivement.

> À noter que le nombre de points `npts` peut être modifié, dans ce qui suit, afin de voir son effet sur la qualité des régressions. Plus il y aura de points, meilleurs seront les résultats.
<!-- #endregion -->



In [ ]:
# intervalles des variables x1 et x2
x1_min, x1_max = 0, 10
x2_min, x2_max = 0, 15

# taille de l'échantillon
npts = 500

# générer l'échantillon
x1 = np.random.uniform(x1_min, x1_max, npts)
x2 = np.random.uniform(x2_min, x2_max, npts)



<!-- #region id="f91382bf" -->
Par la suite, générons le signal théorique $y = a_{0} + a_{1}x_{1} + a_{2}x_{2} + a_{3}x_{1}x_{2}$.
<!-- #endregion -->



In [ ]:
# paramètres réels du modèle
a = [0.10, 0.15, 0.20, 0.50]

# valeur du signal théorique
y = a[0] + a[1] * x1 + a[2] * x2 + a[3] * x1 * x2



<!-- #region id="3a577e29" -->
Ajoutons maintenant le bruit gaussien afin de simuler des données expérimentales.


> À noter que le niveau de bruit dans les données, qui est contrôlé par l'écart type du bruit $\sigma$, peut être modifié afin de voir son effet sur la qualité des régressions. Les performances des régressions **diminuent** lorsque le niveau de bruit **augmente**.

<!-- #endregion -->



In [ ]:
# écart type du bruit gaussien
sigma = 5

# ajouter le bruit gaussien au signal
y = y + np.random.normal(0.0, sigma, npts)



<!-- #region id="aef40706" -->
Maintenant, effectuons la génération de la matrice des données $X$.

On ne génère pas ici la première colonne remplie de 1 de la matrice $X$, car la classe
[LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
de Scikit-learn (utilisée plus loin) la génère automatiquement.
<!-- #endregion -->



In [ ]:
X = np.vstack([x1, x2, (x1 * x2)]).T



<!-- #region id="5645903c" -->
Finalement, séparons les données en deux ensembles&puncsp;: les données d'entraînement ($80~\%$) et les données de test ($20~\%$).

Les données d'entraînement vont servir à entraîner le modèle de régression, c'est-à-dire à estimer les valeurs des paramètres $a_0, a_1, a_2$ et $a_3$. Les données de test vont ensuite servir à mesurer les performances du modèle entraîné à prédire la réponse $y$ sans bruit.

Dans la cellule ci-dessous, effectuez la séparation des données d'entraînement et des données de tests à l'aide de la fonction [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) en complétant l'énoncé avec les arguments de la fonction.

<!-- #endregion -->



In [ ]:
# séparer les données en deux ensembles: entraînement et test
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=seed,
)



<!-- #region id="96925192" -->
## Entraînement du modèle de régression
<!-- #endregion -->

<!-- #region id="92c43db1" -->
Débutons par l'estimation des valeurs des coefficients à partir des données d'entraînement. Pour cela, nous allons entraîner un modèle de régression linéaire ([`LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)) de Scikit-learn avec les paramètres par défaut de la librairie.

Dans la cellule ci-dessous, compléter les énoncés afin de créer un modèle et d'effectuer l'ajustement de ses paramètres avec les données d'entraînement (`X_train` et `y_train`).

<!-- #endregion -->



In [ ]:
# créer un modèle linéaire
reg = linear_model.LinearRegression()

# ajuster de ses paramètres en utilisant les données d'entraînement
reg.fit(X_train, y_train)



<!-- #region id="e3fe7240" -->
On affiche maintenant la valeur des coefficients&puncsp;:
<!-- #endregion -->



In [ ]:
print("Valeurs des coefficients:\n")
print(f"\ta0 vrai: {a[0]:0.2f} \ta0 estimé: {reg.intercept_:0.2f}")
print(f"\ta1 vrai: {a[1]:0.2f} \ta1 estimé: {reg.coef_[0]:0.2f}")
print(f"\ta2 vrai: {a[2]:0.2f} \ta2 estimé: {reg.coef_[1]:0.2f}")
print(f"\ta3 vrai: {a[3]:0.2f} \ta3 estimé: {reg.coef_[2]:0.2f}")



<!-- #region id="b0b97a8c" -->
Les coefficients vrais et mesurés sont parfois similaires et parfois assez différents. On observerait une meilleure concordance entre eux en réduisant le niveau de bruit $\sigma$ dans les données expérimentales (jouez avec la valeur de la variable `sigma` pour voir). Il est à noter que la classe `LinearRegression` de Scikit-learn ne fournit pas les intervalles de confiance des coefficients.
<!-- #endregion -->

<!-- #region id="cff1cff7" -->
### Calcul de la métrique $R^2$ pour les données d'entraînement et de test

Cette métrique est universellement utilisée pour mesurer les performances en régression. Elle est décrite dans le module sur les métriques de qualité en régression. En gros, elle mesure la fraction de l'information contenue dans les données qui est expliquée par le modèle. Une valeur de $R^2 =1$ implique que le modèle prédit exactement les valeurs de $y$. Une valeur de $R^2 =0$ implique que le modèle est incapable de prédire les valeurs de $y$. Elle prend des valeurs entre 0 et 1. Plus elle est près de 1, meilleure est l'adéquation entre le modèle et les données.
<!-- #endregion -->

<!-- #region id="06cf4069" -->
Dans la cellule ci-dessous, complétez les énoncés afin d'appliquer le modèle de régression sur les données d'entraînement et de test, et de calculer les scores $R^2$ pour les données d'entraînement et de test.

Vous devez utiliser le modèle `reg` que nous avons entraîné afin de prédire le signal $y$ pour les données d'entraînement (`X_train`) et de tests (`X_test`).

Ensuite, appelez la fonction [r2_score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.r2_score.html?highlight=r2_score#sklearn.metrics.r2_score)  de Scikit-learn pour calculer et afficher les scores $R^2$ correspondant.
<!-- #endregion -->



In [ ]:
# appliquer le modèle sur les données d'entraînement et de test
y_train_pred = reg.predict(X_train)
y_test_pred = reg.predict(X_test)

# calculer le score pour les données d'entraînement
score = r2_score(y_train, y_train_pred)
print("Valeur de R2 en entraînement:", round(100*score, 1), "%")

# calculer le score pour les données de test
score = r2_score(y_test, y_test_pred)
print("Valeur de R2 en test:", round(100*score, 1), "%")



<!-- #region id="329fe91c" -->
On constate que les résultats sur les données d'entraînement sont supérieurs à ceux des données de test, ce qui est tout à fait normal. Les résultats sur les données de tests sont une meilleure estimation des performances réelles du modèle. Les résultats sur les données d'entraînement sont toujours surestimés par le fait que ce sont elles qui ont servi à estimer les paramètres du modèle.
<!-- #endregion -->

<!-- #region id="b0827b42" -->
## Affichage des résultats
<!-- #endregion -->

<!-- #region id="ad2b837d" -->
Dans ce qui suit, nous allons voir dans quelle mesure le modèle entraîné reproduit bien la distribution des
données d'entraînement et prédit bien les données de test.

Nous débutons par générer une grille de valeurs $(u,v)$ couvrant la plage des valeurs de $x_{1}$ et $x_{2}$.
<!-- #endregion -->



In [ ]:
u, v = np.meshgrid(
    np.linspace(x1_min, x1_max, 30),
    np.linspace(x2_min, x2_max, 30),
)



<!-- #region id="49c69b18" -->
Par la suite, nous générons la matrice des données $X$ pour tous les points de la grille.
<!-- #endregion -->



In [ ]:
X_grille = np.vstack([u.ravel(), v.ravel(), (u * v).ravel()]).T



<!-- #region id="e61090cc" -->
Nous effectuons ensuite la prédiction de la valeur de la réponse $y$ sur la grille.
<!-- #endregion -->



In [ ]:
w = reg.predict(X_grille)
w = w.reshape(u.shape)



<!-- #region id="baac93d1" -->
Finalement, nous affichons les données d'entraînement $(x_{1},x_{2})$ et de la réponse $y = f(x_{1},x_{2})$.
<!-- #endregion -->



In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection="3d")

ax.plot_surface(
    u, v, w, alpha=0.4, color=[1, 1, 0], shade=True, linewidth=0, antialiased=False
)

ax.scatter(x1, x2, y, c="k", s=10)

ax.set_xlabel("$x_{1}$", fontsize=18)
ax.set_ylabel("$x_{2}$", rotation=0, fontsize=18)
ax.set_zlabel("$y$", fontsize=18)
ax.set_title("Modèle linéaire avec interaction", fontsize=18)

ax.set_facecolor("#BBBBFF")
ax.grid(False)



<!-- #region id="7eceabf9" -->
La figure ci-dessus montre que la surface $y=f(x_{1}, x_{2})$ reproduit bien la distribution des données d'entraînement.
On remarque également l'interaction entre les valeurs de $x_{1}$ et $x_{2}$ qui produit la courbure de la surface.

Comparons maintenant les réponses prédites et mesurées avec les données de test. La figure suivante montre la
relation entre elles.
<!-- #endregion -->



In [ ]:
# utiliser les couleurs par défaut de seaborn
sns.set(color_codes=True)

# tracer le graphique
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)

ax.plot(y_test_pred, y_test, "o", color="black", markersize=5)
ax.plot(y_test, y_test, color="yellow", linewidth=2)
ax.set_xlabel("$y_{test}$ prédite", fontsize=18)
ax.set_ylabel("$y_{test}$ vraie", fontsize=18)
ax.set_title("Valeurs de test", fontsize=18)
ax.set_facecolor("#BBBBFF")



<!-- #region id="f311a23b" -->
Les données sont bien distribuées le long de la droite identité en jaune. Il y a une bonne adéquation entre les valeurs prédites et mesurées. C'est d'ailleurs ce qu'indiquait la valeur de $R_{test}^2$.

Pour mieux saisir l'impact du bruit dans la qualité du modèle, essayez différentes valeurs pour la variable `sigma`. Si vous annulez le bruit, la modélisation sera parfaite. De même, expérimentez avec le nombre d'échantillons en modifiant la valeur de la variable `npts`. Jouez aussi avec la valeur de l'argument `test_size` pour la fonction `train_test_split`.
<!-- #endregion -->
